# CLUSTER ASSIGNMENT


## Problem Statement:
HELP International is an international humanitarian NGO that is committed to fighting poverty and proNow the CEO of the NGO needs to decide how to use this money strategically and effectively.

The significant issues that come while making this decision are mostly related to choosing the countries that are in the direst need of aid. viding the people of backward countries with basic amenities and relief during the time of disasters and natural calamities.Now the CEO of the NGO needs to decide how to use this money strategically and effectively.

The significant issues that come while making this decision are mostly related to choosing the countries that are in the direst need of aid. 

**STEPS**
 - Reading and Understanding the Data
 - Data Preparation
 - Finding the optimal number of cluster
 - cluster profiling

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**LIBRARIES**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
import sklearn 
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
import numpy as np
from math import isnan
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

# Reading and Understanding the Data

In [ ]:
df=pd.read_csv('/kaggle/input/help-international-data/Country-data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

**converting percentage values of health,exports,imports into actual values**

In [ ]:
df['exports']=df['exports']*df['gdpp']/100
df['exports']

In [ ]:
df['health']=df['health']*df['gdpp']/100
df['health']

In [ ]:
df['imports']=df['imports']*df['gdpp']/100
df['imports']

# UNIVARIATE ANALYSIS

In [ ]:
# Distplot of all columns
plt.figure(figsize=(15,10))
feature=df.columns[1:]
for i in enumerate(feature):
    plt.subplot(3,3,i[0]+1)
    sns.distplot(df[i[1]])

In [ ]:
# Box plot 
plt.figure(figsize=(15,12))
feature=df.columns[1:]
for i in enumerate(feature):
    plt.subplot(3,3,i[0]+1)
    sns.boxplot(df[i[1]])

**Inferences:As we have to select the countries that are in the direst need of aid.here, child_mort and life_expec we don't touch the upper limit  and in gdpp and income we don't touch the lower limit respectively as we have less number of data we can't remove the outliers and we also dont cap beacuse it is important for our analysis**

# BIVARIATE ANALYSIS

In [ ]:
sns.scatterplot(x='income',y='gdpp',data=df)
plt.xlabel('income')
plt.ylabel('gdpp')
plt.show()

**inferernces:countries having low income have low gdp**

In [ ]:
sns.scatterplot(x='child_mort',y='income',data=df)
plt.xlabel('child_mort')
plt.ylabel('income')
plt.show()

**inferernces:countries having low income have high child mortality**

In [ ]:
sns.scatterplot(x='child_mort',y='gdpp',data=df)
plt.xlabel('child_mort')
plt.ylabel('gdpp')
plt.show()

**inferernces:countries having high child mortality have low gdp**

In [ ]:
# Outliers treatment
Q1=df.exports.quantile(.05)
Q3=df.exports.quantile(.75)
IQR=Q3-Q1
df=df[(df.exports>= Q1) & (df.exports<= Q3+1.5*IQR)]

Q1=df.imports.quantile(.05)
Q3=df.imports.quantile(.95)
IQR=Q3-Q1
df=df[(df.imports>= Q1) & (df.imports<= Q3+1.5*IQR)]

Q1=df.life_expec.quantile(.01)
Q3=df.life_expec.quantile(.75)
IQR=Q3-Q1
df=df[(df.life_expec>= Q1-1.5*IQR) & (df.life_expec<= Q3+1.5*IQR)]

In [ ]:
Q1=df.child_mort.quantile(.01)
Q3=df.child_mort.quantile(.99)
IQR=Q3-Q1
df=df[(df.child_mort>= Q1-1.5*IQR) & (df.child_mort<= Q3+1.5*IQR)]

Q1=df.income.quantile(.01)
Q3=df.income.quantile(.75)
IQR=Q3-Q1
df=df[(df.income>= Q1) & (df.income<= Q3+1.5*IQR)]

Q1=df.gdpp.quantile(.01)
Q3=df.gdpp.quantile(.75)
IQR=Q3-Q1
df=df[(df.gdpp>= Q1) & (df.gdpp<= Q3+1.5*IQR)]


In [ ]:
#country having low GDP
df.groupby(['country'])['gdpp'].max().sort_values(ascending=True)

In [ ]:
#country with high child mortality 
df.groupby(['country'])['child_mort'].max().sort_values(ascending=False)

In [ ]:
clust=df.groupby(['country'])['child_mort'].mean()
clust=clust.reset_index()
clust.head()

In [ ]:
income=df.groupby(['country'])['income'].mean()
income=income.reset_index()
income.head()

In [ ]:
gdpp=df.groupby(['country'])['gdpp'].mean()
gdpp=gdpp.reset_index()
gdpp.head()

In [ ]:
life_expec=df.groupby(['country'])['life_expec'].mean()
life_expec=life_expec.reset_index()
life_expec.head()

In [ ]:
health=df.groupby(['country'])['health'].mean()
health=health.reset_index()
health.head()

In [ ]:
clust=pd.merge(clust,income,on='country',how='inner')
clust.head()

In [ ]:
clust=pd.merge(clust,gdpp,on='country',how='inner')
clust.head()

In [ ]:
clust=pd.merge(clust,life_expec,on='country',how='inner')
clust.head()

In [ ]:
#top 5 variables for clustring
clust=pd.merge(clust,health,on='country',how='inner')
clust.head()

In [ ]:
#scaling
clust_df=clust[['child_mort','income','gdpp']]
scaler=StandardScaler()
clust_scaled=scaler.fit_transform(clust_df)
clust_scaled.shape 

In [ ]:
#coverting numpy into dataframe
clust_scaled= pd.DataFrame(clust_scaled)
clust_scaled.columns = ['child_mort','income','gdpp']

In [ ]:
def hopkins(X):
    d = X.shape[1]
    #d = len(vars) # columns
    n = len(X) # rows
    m = int(0.1 * n) 
    nbrs = NearestNeighbors(n_neighbors=1).fit(X.values)
 
    rand_X = sample(range(0, n, 1), m)
 
    ujd = []
    wjd = []
    for j in range(0, m):
        u_dist, _ = nbrs.kneighbors(uniform(np.amin(X,axis=0),np.amax(X,axis=0),d).reshape(1, -1), 2, return_distance=True)
        ujd.append(u_dist[0][1])
        w_dist, _ = nbrs.kneighbors(X.iloc[rand_X[j]].values.reshape(1, -1), 2, return_distance=True)
        wjd.append(w_dist[0][1])
 
    H = sum(ujd) / (sum(ujd) + sum(wjd))
    if isnan(H):
        print(ujd, wjd)
        H = 0
 
    return H

In [ ]:
#Use the Hopkins Statistic function by passing the above dataframe as a paramter
hopkins(clust_scaled)

# K-MEANS CLUSTERING

In [ ]:
#elbow curve
ssd=[]
range_clusters=[2,3,4,5,6,7,8]

for num_clusters in range_clusters:
    kmeans=KMeans(n_clusters=num_clusters,max_iter=50)
    kmeans.fit(clust_scaled)
    ssd.append(kmeans.inertia_)
    
plt.plot(ssd)

In [ ]:
#silhoue score
range_clusters=[2,3,4,5,6,7,8]

for num_clusters in range_clusters:
    
    #intialise kmeans
    kmeans=KMeans(n_clusters=num_clusters,max_iter=50)
    kmeans.fit(clust_scaled)
    
   
    cluster_labels=kmeans.labels_
    
     #silhouette score
    silhouette_avg=silhouette_score(clust_scaled,cluster_labels)
    print("For n_clusters={0}, the silhouette score is{1}".format(num_clusters,silhouette_avg))

In [ ]:
#final model K=3
kmeans=KMeans(n_clusters=3,max_iter=50)
kmeans.fit(clust_scaled)

In [ ]:
kmeans.labels_

In [ ]:
#assign cluster
clust['cluster_id']=kmeans.labels_
clust.head()

In [ ]:
sns.scatterplot(x='income',y='child_mort',data=clust,hue='cluster_id',)
plt.show()

In [ ]:
sns.scatterplot(x='income',y='gdpp',data=clust,hue='cluster_id',)
plt.show()

In [ ]:
sns.scatterplot(x='gdpp',y='child_mort',data=clust,hue='cluster_id',)
plt.show()

# CLUSTER PROFILING

In [ ]:
sns.boxplot(x='cluster_id',y='child_mort',data=clust)
plt.show()

In [ ]:
sns.boxplot(x='cluster_id',y='income',data=clust)
plt.show()

In [ ]:
sns.boxplot(x='cluster_id',y='gdpp',data=clust)
plt.show()

In [ ]:
#low GDP, Low income,High Child mortality
Top_countries=clust[clust['cluster_id']==2].sort_values(by=['gdpp','income','child_mort'],ascending=['False','False','True'])
Top_countries

In [ ]:
Top_countries.head() #kmeans

# Hierarchical clustering

In [ ]:
mergings=linkage(clust_scaled,method='single',metric='euclidean')
dendrogram(mergings)
plt.show()

In [ ]:
mergings=linkage(clust_scaled,method='complete',metric='euclidean')
dendrogram(mergings)
plt.show()

In [ ]:
cluster_labels=cut_tree(mergings,n_clusters=3).reshape(-1,)
cluster_labels.shape

In [ ]:
clust['cluster_labels']=cluster_labels
clust.head()

# CLUSTER PROFILING

In [ ]:
sns.boxplot(x='cluster_labels',y='child_mort',data=clust)
plt.show()

In [ ]:
sns.boxplot(x='cluster_labels',y='income',data=clust)
plt.show()

In [ ]:
sns.boxplot(x='cluster_labels',y='gdpp',data=clust)
plt.show()

In [ ]:
#low GDP, Low income,High Child mortality
Top_countries1=clust[clust['cluster_labels']==1].sort_values(by=['gdpp','income','child_mort'],ascending=['False','False','True'])
Top_countries1

In [ ]:
Top_countries1.head() #hierarchical cluster

**INFERENCES:WE GOT TOP 5 COUNTRIES FROM BOTH K-MEANS AND HIERARCHICAL CLUSTERING.HAITI ,MALI AND BENIN BOTH ARE IN COMMON**